In [2]:
import math
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import csv
%matplotlib inline
import time
from tqdm import tqdm
import multiprocessing as mp
import pandas as pd
import networkx as nx
import scipy.stats as st

In [3]:
def Boolean_f(interact_net,cal_type,input_state):
    ##cal_type=1时为并运算，为2时为合运算
    nodes_num=len(cal_type)
    S_temp=np.zeros(nodes_num)
  
    for i in range(2):
        if cal_type[i]>0:
            S_temp[i]=input_state[i]
    for i in range(nodes_num):
        for j in range(nodes_num):
            S_temp[i] +=interact_net[j,i]*input_state[j]
    
    for i in range(nodes_num):
        if cal_type[i]==1:
            if S_temp[i]>1:
                S_temp[i]=1
            elif S_temp[i]<0:
                S_temp[i]=0
        elif cal_type[i]==2:
            if S_temp[i]<=2:
                S_temp[i]=0
            elif S_temp[i]==3:
                S_temp[i]=1
        elif cal_type[i]==0:
            S_temp[i]=0
    return S_temp

In [4]:
###给定函数的2次数，输出二进制状态函数
def Init_boolState(st_id,nodes_num):
    bin_obj = bin(int(st_id))[2:]
    bin_str = bin_obj.rjust(nodes_num,'0')
    o_arr = []
    for ii in range(len(bin_str)):
        o_arr.append(int(bin_str[len(bin_str)-ii-1]))
    return o_arr    

In [5]:
def Stable_f(ini_state,interact_net,cal_type):
    S1=Boolean_f(interact_net,cal_type,ini_state)
    S2=Boolean_f(interact_net,cal_type,S1)
    if (S2==S1).all():
        return S2
    else:
        t=0
        while (S2!=S1).any() and t<100:
            S1=Boolean_f(interact_net,cal_type,S2)
            S2=Boolean_f(interact_net,cal_type,S1)
            t+=1
        return S2

In [6]:
def State2num(input_state,nodes_num):
    num=0
    for a in range(nodes_num):
        num+=input_state[a]*(2**a)
    return(num)

In [7]:
def trans221(input_net):
    node_num=len(input_net)
    output_net=np.zeros([node_num,4])
    for i in range(node_num):
        output_net[i,0]=i
        output_net[i,2]=sum(input_net[:,i])
        for j in range(node_num):
            if input_net[i,j]>0:
                output_net[i,1]=j
                stat=Init_boolState(j,9)
                if stat[0]==0 and stat[1]==0:
                    output_net[i,3]=0
                elif stat[0]==1 and stat[1]==0:
                    output_net[i,3]=1
                elif stat[0]==0 and stat[1]==1:
                    output_net[i,3]=100
                elif stat[0]==1 and stat[1]==1:
                    output_net[i,3]=2
    return(output_net)

In [8]:
def Traj(nodes_num,matrix_interact,cal_type):
    state_trans=np.zeros([2**nodes_num,2**nodes_num])
    nodes_si=np.zeros(nodes_num)
    nodes_stemp1=np.zeros(nodes_num)
    nodes_stemp2=np.zeros(nodes_num)
    for i in range(2**nodes_num):
        nodes_si=Init_boolState(i,nodes_num)
        nodes_stemp1=nodes_si
        nodes_stemp2=Boolean_f(matrix_interact,cal_type,nodes_stemp1)
        state_trans[i,int(State2num(nodes_stemp2,nodes_num))]=1
    trajec=trans221(state_trans)
    return trajec

In [9]:
##Dynamics in the standard simplified networks
matrix_interact = np.loadtxt('PLEASE TYPE IN THE FILE ADDRESS\\nodes_interaction_G.csv', delimiter=';')
cal_type = np.ones(11)
cal_type[5] = 2
trajec0=Traj(11,matrix_interact,cal_type)
data0 = pd.DataFrame(trajec0)
data0.to_csv('nG_state_0_traj.csv')

In [11]:
##DELETE CaM
matrix_koCM = np.loadtxt('PLEASE TYPE IN THE FILE ADDRESS\\nodesG_intc_koCM.csv', delimiter=';')
cal_type_CM = np.ones(10)
trajCM=Traj(10,matrix_koCM,cal_type_CM)
dataTJCM = pd.DataFrame(trajCM) 
dataTJCM.to_csv('nG_traj_koCM.csv')

##DELETE CaN
matrix_koCN = np.loadtxt('PLEASE TYPE IN THE FILE ADDRESS\\nodesG_intc_koCN.csv', delimiter=';')
cal_type_CN = np.ones(10)
cal_type_CN[5] = 2
trajCN=Traj(10,matrix_koCN,cal_type_CN)
dataTJCN = pd.DataFrame(trajCN) 
dataTJCN.to_csv('nG_traj_koCN.csv')



##DELETE PKC
matrix_koC = np.loadtxt('PLEASE TYPE IN THE FILE ADDRESS\\nodesG_intc_koC.csv', delimiter=';')
cal_type_C = np.ones(10)
trajC=Traj(10,matrix_koC,cal_type_C)
dataTJC = pd.DataFrame(trajC) 
dataTJC.to_csv('nG_traj_koC.csv')

##DELETE NSF
matrix_koNSF = np.loadtxt('PLEASE TYPE IN THE FILE ADDRESS\\nodesG_intc_koNSF.csv', delimiter=';')
cal_type_NSF = np.ones(10)
cal_type_NSF[4]=2
trajNSF=Traj(10,matrix_koNSF,cal_type_NSF)
dataTJNSF = pd.DataFrame(trajNSF) 
dataTJNSF.to_csv('nG_traj_koNSF.csv')

##DELETE PSD95 or TARP
matrix_koPSD = np.loadtxt('PLEASE TYPE IN THE FILE ADDRESS\\nodesG_intc_koPSD.csv', delimiter=';')
state_transPSD=np.zeros([1024,1024])
cal_type_PSD = np.ones(10)
cal_type_PSD[5]=2
trajPSD=Traj(10,matrix_koPSD,cal_type_PSD)
dataTJPSD = pd.DataFrame(trajPSD) 
dataTJPSD.to_csv('nG_traj_koPSD.csv')

##DELETE CaMKII
matrix_koCMK = np.loadtxt('PLEASE TYPE IN THE FILE ADDRESS\\nodesG_intc_koCMK.csv', delimiter=';')
state_transCMK=np.zeros([1024,1024])
cal_type_CMK=np.ones(10)
cal_type_CMK[5]=2
trajCMK=Traj(10,matrix_koCMK,cal_type_CMK)
dataTJCMK = pd.DataFrame(trajCMK) 
dataTJCMK.to_csv('nG_traj_koCMK.csv')


##DELETE GRIP
matrix_koGRIP = np.loadtxt('PLEASE TYPE IN THE FILE ADDRESS\\nodesG_intc_koGRIP.csv', delimiter=';')
state_transGRIP=np.zeros([1024,1024])
cal_type_GRIP = np.ones(10)
cal_type_GRIP[4]=2
trajGRIP=Traj(10,matrix_koGRIP,cal_type_GRIP)
dataTJGRIP = pd.DataFrame(trajGRIP) 
dataTJGRIP.to_csv('nG_traj_koGRIP.csv')


##DELETE PP1
matrix_koP1 = np.loadtxt('PLEASE TYPE IN THE FILE ADDRESS\\nodesG_intc_koP1.csv', delimiter=';')
state_transP1=np.zeros([2048,2048])
cal_type_P1 = np.ones(11)
cal_type_P1[5]=2
trajP1=Traj(11,matrix_koP1,cal_type_P1)
dataTJP1 = pd.DataFrame(trajP1) 
dataTJP1.to_csv('nG_traj_koP1.csv')

##DELETE nos
matrix_koNOS = np.loadtxt('PLEASE TYPE IN THE FILE ADDRESS\\nodesG_intc_koNOS.csv', delimiter=';')
state_transNOS=np.zeros([2048,2048])
cal_type_NOS = np.ones(11)
cal_type_NOS[5]=2
trajNOS=Traj(11,matrix_koNOS,cal_type_NOS)
dataTJNOS = pd.DataFrame(trajNOS) 
dataTJNOS.to_csv('nG_traj_koNOS.csv')


##DELETE SAP97/102
matrix_koS97 = np.loadtxt('PLEASE TYPE IN THE FILE ADDRESS\\nodesG_intc_koS97.csv', delimiter=';')
cal_type_S97 = np.ones(11)
cal_type_S97[5]=2
trajS97=Traj(11,matrix_koS97,cal_type_S97)
dataTJS97 = pd.DataFrame(trajS97) 
dataTJS97.to_csv('nG_traj_koS97.csv')

##DELETE IP3
matrix_koI3 = np.loadtxt('PLEASE TYPE IN THE FILE ADDRESS\\nodesG_intc_koIP3.csv', delimiter=';')
cal_type_I3 = np.ones(11)
cal_type_I3[5]=2
trajI3=Traj(11,matrix_koI3,cal_type_I3)
dataTJI3 = pd.DataFrame(trajI3) 
dataTJI3.to_csv('nG_traj_koI3.csv')



In [12]:
##summarizaton of betweenness
def btn_sum(data):
    n = len(data)
    d=0
    p=0
    btn_totl=np.zeros(2)
    G_P0=nx.DiGraph()
    G_D0=nx.DiGraph()
    for i in range(n):
        if data[i,3]==1:
            G_D0.add_edges_from([(data[i,0],data[i,1])])
            d+=1
        elif data[i,3]==2:
            G_P0.add_edges_from([(data[i,0],data[i,1])])
            p+=1
    btn_P = nx.betweenness_centrality(G_P0,endpoints=True,normalized=False)
    btn_D = nx.betweenness_centrality(G_D0,endpoints=True,normalized=False)
    btn_totl[0]=sum(btn_P.values())
    btn_totl[1]=sum(btn_D.values())
    return btn_totl
    

In [15]:
##统计节点介数
btn_totl=np.zeros([2,12])
#btn_totl[0,:]为P,[1,:]为D，排序与caltype一样
btn_totl[:,0]=btn_sum(trajec0)
btn_totl[:,1]=btn_sum(trajCM)
btn_totl[:,2]=btn_sum(trajCN)
btn_totl[:,3]=btn_sum(trajI3)
btn_totl[:,4]=btn_sum(trajNOS)
btn_totl[:,5]=btn_sum(trajC)
btn_totl[:,6]=btn_sum(trajCMK)
btn_totl[:,7]=btn_sum(trajS97)
btn_totl[:,8]=btn_sum(trajNSF)
btn_totl[:,9]=btn_sum(trajPSD)
btn_totl[:,10]=btn_sum(trajP1)
btn_totl[:,11]=btn_sum(trajGRIP)

##标记功能缺失的节点损伤
btn_totl[0,1]=0
btn_totl[1,1]=0
btn_totl[1,2]=0
btn_totl[1,3]=0
btn_totl[1,5]=0
btn_totl[0,6]=0
btn_totl[0,7]=0
btn_totl[0,8]=0
btn_totl[1,10]=0

databtn_totl = pd.DataFrame(btn_totl) 
databtn_totl.to_csv('btn_totl.csv')

KO_loss = np.zeros([2,11])
for i in range(11):
    KO_loss[0,i]=1-btn_totl[0,i+1]/btn_totl[0,0]
    KO_loss[1,i]=1-btn_totl[1,i+1]/btn_totl[1,0]
    if KO_loss[0,i]<0:
        KO_loss[0,i]=0
    if KO_loss[1,i]<0:
        KO_loss[1,i]=0
databtn_KO_loss = pd.DataFrame(KO_loss) 
databtn_KO_loss.to_csv('KO_loss.csv')

KO_loss_totl = np.zeros(11)
for i in range(11):
    KO_loss_totl[i]=math.sqrt(KO_loss[0,i]**2+KO_loss[1,i]**2) 

x = KO_loss_totl
y = np.array([1,0.625,0.5,0.4,0.4444444,0.2,0.166666667,0.66666667,0.25,0.5,0.2])
slope, intercept,r_value,p_value, std_err = st.linregress(x,y)

print(r_value)
print(p_value)

0.7120454048982501
0.013964673506096674


In [ ]:
##TARP or PSD95 overexpression
matrix_oePSD = np.loadtxt('PLEASE TYPE IN THE FILE ADDRESS\\nodes_intc_oePSD.csv', delimiter=';')
state_transPSD=np.zeros([512,512])
cal_type_PSD = np.ones(9)
nodes_siPSD=np.zeros(9)
cal_type_PSD[3] = 2
nodes_siPSD[7]=1
for i in range(512):
    nodes_siPSD=Init_boolState(i,9)
    nodes_ssPSD=Stable_f(nodes_siPSD,matrix_oePSD,cal_type_PSD)
    stable_id=int(State2num(nodes_ssPSD,9))
    state_transPSD[i,stable_id]=1
dataTAR = pd.DataFrame(state_transPSD)
dataTAR.to_csv('n_state_oePSD.csv')

##CaMKII overexpression
matrix_oeCMK = np.loadtxt('PLEASE TYPE IN THE FILE ADDRESS\\nodes_intc_oeCMK.csv', delimiter=';')
state_transCMK=np.zeros([512,512])
cal_type_CMK = np.ones(9)
nodes_siCMK=np.zeros(9)
cal_type_CMK[3] = 2
nodes_siCMK[6]=1
for i in range(512):
    nodes_siCMK=Init_boolState(i,9)
    nodes_ssCMK=Stable_f(nodes_siCMK,matrix_oeCMK,cal_type_CMK)
    stable_id=int(State2num(nodes_ssCMK,9))
    state_transCMK[i,stable_id]=1
dataCMK = pd.DataFrame(state_transCMK)
dataCMK.to_csv('n_state_oeCMK.csv')